In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
from scipy.spatial import distance

In [2]:
db_filename = 'dinofunworld.db'
#conn = sqlite3.connect("./readonly/{0}".format(db_filename)) 
conn = sqlite3.connect(db_filename) 
c = conn.cursor()

In [3]:
def calculateDisSimilarity(firstSequenceSet, secondSequenceSet):
    returnValue = []
    return reduce(lambda a,b: a+b,
                  [0 if firstRecord == secondRecord else 1 
                       for firstRecord, secondRecord in zip(firstSequenceSet, secondSequenceSet)]
                 )

In [7]:
visitorSequencesGiven = [165316, 1835254, 296394, 404385, 448990]
visitorSequencesGivenString = list(map(str, visitorSequencesGiven))
visitorSequenceInQuery = ",".join(visitorSequencesGivenString)
sequences = c.execute("SELECT visitorID,sequence FROM sequences WHERE visitorID IN ({0})".format(visitorSequenceInQuery)).fetchall()
sequencesDataFrame = pd.DataFrame.from_records(sequences, columns=['VisitorID','SequenceValue'], index='VisitorID')

convertedValues = [list(map(int, x.split('-'))) for x in sequencesDataFrame.SequenceValue]
sequencesDataFrame['convertedSequenceValues'] = convertedValues
finalValues = {}
print(sequencesDataFrame)
for x,y in sequencesDataFrame.iterrows():
    rowValues = {}
    for a,b in sequencesDataFrame.iterrows():
        if x != a:
            rowValues[a] = calculateDisSimilarity(y['convertedSequenceValues'], b['convertedSequenceValues'])
    finalValues[x] = rowValues

print(finalValues)


                                               SequenceValue  \
VisitorID                                                      
165316     0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...   
296394     0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...   
404385     0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...   
448990     0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...   
1835254    0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-...   

                                     convertedSequenceValues  
VisitorID                                                     
165316     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
296394     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
404385     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
448990     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1835254    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
{165316: {296394: 318, 404385: 329, 448990: 318, 1835254: 336}, 296394: {165316: 318, 404385: 303, 448990: 0, 

In [5]:
''' Queries the database for a specific category name in category 
    table and returns {name, attractionID} from each of the rows 
    satisfying condition. categoryName is filtered using %categoryName%'''
def getRideAttractionIDsAndNames(categoryName):
    return c.execute("SELECT Name, AttractionID FROM attraction WHERE Category like '%{0}%'".format(categoryName)).fetchall()

In [6]:
# Graded Cell, PartID: 8S2jm
# Create and display a Parallel Coordinate Plot displaying the minimum, average, 
# and maximum attendance for each ride in the park (note that not all attractions
# are rides).
